# Improved RAG System for A Christmas Carol

This notebook implements an **improved** RAG (Retrieval-Augmented Generation) system with the following enhancements:

## Key Improvements:

1. **Skip Reformulation** - Search directly with original question (simpler, faster)
2. **Increased Context** - Retrieve 5-10 chunks instead of 3
3. **Better Chunking** - Use 512 character overlap (was 50) for better context preservation
4. **Persistent Storage** - Use ChromaDB PersistentClient to avoid re-embedding
5. **Improved Prompt** - Add instructions for handling "unknown" cases
6. **Larger Model** - Use FLAN-T5-large (780M params vs 250M)
7. **Reranking** - Add cross-encoder reranking for better relevance
8. **Source Attribution** - Show chunk IDs and metadata with answers

In [2]:
# Import libraries
import os
from langchain_community.document_loaders import UnstructuredEPubLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

import chromadb
from uuid import uuid4
from chromadb.utils import embedding_functions

from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, AutoModelForSequenceClassification
import torch

ModuleNotFoundError: No module named 'chromadb'

## Improvement #3: Better Chunking Strategy

**Changed from:**
- chunk_size = 1024
- chunk_overlap = 50 (only 5%!)

**Changed to:**
- chunk_size = 1024 (same)
- chunk_overlap = 512 (50% overlap)

**Why this matters:**
- 512 character overlap ensures important context at chunk boundaries is preserved
- Prevents conversations/paragraphs from being split awkwardly
- Each chunk shares significant context with neighbors
- Improves retrieval quality when answer spans multiple chunks

In [3]:
# Load document with improved chunking
chunk_size = 1024
chunk_overlap = 512  # IMPROVED: 50% overlap instead of 5%
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=chunk_size, 
    chunk_overlap=chunk_overlap
)

epub_loader = UnstructuredEPubLoader('./docs/charles-dickens_a-christmas-carol.epub')

In [4]:
# Split document
chunks = epub_loader.load_and_split(text_splitter)
print(f"Number of chunks: {len(chunks)}")
print(f"\nSample chunk:\n{chunks[100]}")

[WARNING] Could not load translations for en-US
  data file translations/en.yaml not found
[WARNING] The term Abstract has no translation defined.



Number of chunks: 274

Sample chunk:
page_content='“My time grows short,” observed the Spirit. “Quick!”

This was not addressed to Scrooge, or to anyone whom he could see, but it produced an immediate effect. For again Scrooge saw himself. He was older now; a man in the prime of life. His face had not the harsh and rigid lines of later years; but it had begun to wear the signs of care and avarice. There was an eager, greedy, restless motion in the eye, which showed the passion that had taken root, and where the shadow of the growing tree would fall.

He was not alone, but sat by the side of a fair young girl in a mourning dress: in whose eyes there were tears, which sparkled in the light that shone out of the Ghost of Christmas Past.

“It matters little,” she said softly. “To you, very little. Another idol has displaced me; and, if it can cheer and comfort you in time to come as I would have tried to do, I have no just cause to grieve.”

“What idol has displaced you?” he rejoined.

“A 

## Create Embeddings

In [5]:
# Create embedding model (same as before - works well)
embed_model_name = "BAAI/bge-small-en-v1.5"
chroma_embed_func = embedding_functions.SentenceTransformerEmbeddingFunction(model_name=embed_model_name)

## Improvement #4: Persistent ChromaDB Storage

**Changed from:**
```python
ch_client = chromadb.Client()  # Ephemeral - lost on restart
```

**Changed to:**
```python
ch_client = chromadb.PersistentClient(path="./chroma_db")  # Saved to disk
```

**Benefits:**
- No need to re-embed documents every time
- Much faster startup after first run
- Production-ready storage
- Can share database across multiple sessions

In [ ]:
# Prepare chunks for ChromaDB
texts = [c.page_content for c in chunks]
text_ids = [str(uuid4())[:8] for _ in range(len(texts))]

# Store chunk metadata for source attribution (Improvement #8)
metadatas = [
    {
        "chunk_id": i,
        "source": chunks[i].metadata.get("source", "unknown"),
        "text_preview": texts[i][:100] + "..."
    }
    for i in range(len(texts))
]

print(f"Total texts: {len(texts)}")
print(f"Total IDs: {len(text_ids)}")
print(f"Sample metadata: {metadatas[0]}")

Total texts: 274
Total IDs: 274
Sample metadata: {'chunk_id': 0, 'source': './docs/charles-dickens_a-christmas-carol.epub', 'text_preview': 'Imprint\n\nThe Standard Ebooks logo.\n\nThis ebook is the product of many hours of hard work by voluntee...'}


## Code Explanation: Preparing Chunks for ChromaDB

This code prepares document chunks for storage in **ChromaDB**, a vector database.

### 1. Extract Text Content
```python
texts = [c.page_content for c in chunks]
```
- Creates a list of just the text content from each chunk
- `chunks` are LangChain `Document` objects (from the EPUB loader)
- `page_content` is the actual text of each chunk

### 2. Generate Unique IDs
```python
text_ids = [str(uuid4())[:8] for _ in range(len(texts))]
```
- Creates a unique 8-character ID for each chunk
- `uuid4()` generates a random UUID (e.g., `"a1b2c3d4-e5f6-..."`)
- `[:8]` takes just the first 8 characters for brevity
- ChromaDB requires unique IDs to identify each document

### 3. Create Metadata for Source Attribution
```python
metadatas = [
    {
        "chunk_id": i,                                          # Position in the list (0, 1, 2...)
        "source": chunks[i].metadata.get("source", "unknown"),  # Original file/source
        "text_preview": texts[i][:100] + "..."                  # First 100 chars preview
    }
    for i in range(len(texts))
]
```
- Creates a metadata dictionary for each chunk containing:
  - **`chunk_id`**: The index number of the chunk
  - **`source`**: Where the chunk came from (e.g., the EPUB filename)
  - **`text_preview`**: A short preview of the text (useful for debugging/display)

### Why This Matters
This metadata enables **source attribution** (Improvement #8) — when the RAG system answers a question, it can show *which chunks* the answer came from, making the system more transparent and verifiable.

In [ ]:
# IMPROVEMENT #4: Create PERSISTENT Chroma client
col_name = 'carol_improved'

ch_client = chromadb.PersistentClient(path="./chroma_db")

# Drop existing collection if it exists
try:
    ch_client.delete_collection(col_name)
    print(f"Deleted existing collection: {col_name}")
except:
    print(f"No existing collection to delete")

# Create collection with metadata
carol_col = ch_client.create_collection(
    name=col_name,
    embedding_function=chroma_embed_func,
    metadata={"description": "A Christmas Carol with improved chunking and metadata"}
)

print(f"Created collection: {col_name}")

Deleted existing collection: carol_improved
Created collection: carol_improved


## Code Explanation: Creating Persistent ChromaDB Collection

This code sets up a **persistent vector database** using ChromaDB.

### 1. Define Collection Name
```python
col_name = 'carol_improved'
```
- Names the collection for easy reference throughout the notebook

### 2. Create Persistent Client
```python
ch_client = chromadb.PersistentClient(path="./chroma_db")
```
- Creates a ChromaDB client that **saves data to disk** at `./chroma_db`
- Unlike `chromadb.Client()` (ephemeral), this persists across sessions
- **Benefit**: No need to re-embed documents when you restart the notebook

### 3. Delete Existing Collection (if exists)
```python
try:
    ch_client.delete_collection(col_name)
except:
    print(f"No existing collection to delete")
```
- Removes any previous version of the collection
- Ensures a clean slate for fresh embeddings
- Uses try/except to handle the case where collection doesn't exist yet

### 4. Create New Collection
```python
carol_col = ch_client.create_collection(
    name=col_name,
    embedding_function=chroma_embed_func,
    metadata={"description": "..."}
)
```
- **`name`**: Unique identifier for this collection
- **`embedding_function`**: The BGE model — ChromaDB will automatically embed any text added to this collection
- **`metadata`**: Optional description for documentation

### Why Persistent Storage Matters
- **Development**: Restart kernel without re-embedding (saves 5-10 minutes)
- **Production**: Database survives application restarts
- **Sharing**: Can copy the `chroma_db` folder to share with others

In [1]:
# Insert documents with metadata
carol_col.add(
    documents=texts,
    ids=text_ids,
    metadatas=metadatas  # IMPROVEMENT #8: Store metadata for source attribution
)

print(f"Collection contains {carol_col.count()} documents")

NameError: name 'carol_col' is not defined

## Test Basic Retrieval

In [10]:
# Test query
query = "What happened to Marley?"

results = carol_col.query(
    query_texts=[query],
    n_results=5
)

print(f"Query: {query}")
print(f"\nTop 5 results:")
for i, (doc_id, distance, doc) in enumerate(zip(
    results['ids'][0], 
    results['distances'][0], 
    results['documents'][0]
)):
    print(f"\n{i+1}. ID: {doc_id}, Distance: {distance:.4f}")
    print(f"   Preview: {doc[:150]}...")

Query: What happened to Marley?

Top 5 results:

1. ID: e6eb8a19, Distance: 0.2571
   Preview: Their faithful Friend and Servant,

C. D.

December, 1843.

Stave I

Marley’s Ghost

Marley was dead, to begin with. There is no doubt whatever about ...

2. ID: 6bca2ca7, Distance: 0.3132
   Preview: Scrooge had often heard it said that Marley had no bowels, but he had never believed it until now.

No, nor did he believe it even now. Though he look...

3. ID: 43eb3f5b, Distance: 0.3197
   Preview: “It’s humbug still!” said Scrooge. “I won’t believe it.”

His colour changed, though, when, without a pause, it came on through the heavy door and pas...

4. ID: e2b05587, Distance: 0.3263
   Preview: Mind! I don’t mean to say that I know of my own knowledge, what there is particularly dead about a doornail. I might have been inclined, myself, to re...

5. ID: 3686ea7c, Distance: 0.3268
   Preview: Standard Ebooks is a volunteer-driven project that produces ebook editions of public domain literatur

## Improvement #6: Use Larger Model (FLAN-T5-Large)

**Changed from:**
- google/flan-t5-base (250M parameters)

**Changed to:**
- google/flan-t5-large (780M parameters)

**Benefits:**
- Better reasoning capabilities
- More accurate answers
- Better instruction following
- Can handle longer contexts

**Note:** If this is too slow/large, you can also try:
- google/flan-t5-xl (3B parameters) for even better quality
- Or keep flan-t5-base if speed is critical

In [ ]:
# IMPROVEMENT #6: Load larger model
model_name = "google/flan-t5-base"  # Was: flan-t5-base

model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

print(f"Loaded model: {model_name}")
print(f"Model parameters: {model.num_parameters() / 1e6:.0f}M")

config.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

Loaded model: google/flan-t5-large
Model parameters: 783M


## Improvement #7: Add Reranking with Cross-Encoder

**What is reranking?**
- Initial retrieval uses bi-encoder (fast but less accurate)
- Reranking uses cross-encoder (slower but more accurate)
- Cross-encoder sees query + document together, not separately
- Better at determining relevance

**Process:**
1. Retrieve top-K chunks (e.g., 10) with bi-encoder
2. Rerank with cross-encoder
3. Use top-N reranked chunks (e.g., 5) for answer generation

**Model used:** `cross-encoder/ms-marco-MiniLM-L-6-v2`
- Fast and effective
- Trained on MS MARCO passage ranking dataset

In [12]:
# IMPROVEMENT #7: Load cross-encoder for reranking
reranker_model_name = "cross-encoder/ms-marco-MiniLM-L-6-v2"
reranker_model = AutoModelForSequenceClassification.from_pretrained(reranker_model_name)
reranker_tokenizer = AutoTokenizer.from_pretrained(reranker_model_name)

print(f"Loaded reranker: {reranker_model_name}")

def rerank_results(query, documents, top_k=5):
    """
    Rerank documents using cross-encoder.
    
    Args:
        query: The search query
        documents: List of document texts
        top_k: Number of top results to return
    
    Returns:
        List of (score, doc_index) tuples, sorted by score descending
    """
    # Create query-document pairs
    pairs = [[query, doc] for doc in documents]
    
    # Tokenize
    inputs = reranker_tokenizer(
        pairs,
        padding=True,
        truncation=True,
        return_tensors="pt",
        max_length=512
    )
    
    # Get scores
    with torch.no_grad():
        scores = reranker_model(**inputs).logits.squeeze(-1)
    
    # Sort by score
    scored_docs = [(score.item(), i) for i, score in enumerate(scores)]
    scored_docs.sort(reverse=True, key=lambda x: x[0])
    
    return scored_docs[:top_k]

config.json:   0%|          | 0.00/794 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/90.9M [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

Loaded reranker: cross-encoder/ms-marco-MiniLM-L-6-v2


## Complete RAG Pipeline with All Improvements

### Changes from Original:

1. **No reformulation step** - Search directly with question
2. **Retrieve 10 chunks** - More context candidates
3. **Rerank to top 5** - Better relevance selection
4. **Improved prompt** - Instructions for unknown cases
5. **Source attribution** - Return chunk IDs and previews

In [13]:
def answer_question_improved(question, retrieve_k=10, use_k=5, show_sources=True):
    """
    Improved RAG pipeline.
    
    Args:
        question: The question to answer
        retrieve_k: Number of chunks to retrieve initially (default: 10)
        use_k: Number of chunks to use after reranking (default: 5)
        show_sources: Whether to show source chunks (default: True)
    
    Returns:
        dict with 'answer', 'sources', and 'confidence'
    """
    
    # IMPROVEMENT #1: Skip reformulation, search with original question
    print(f"Question: {question}\n")
    
    # IMPROVEMENT #2: Retrieve more chunks (10 instead of 3)
    print(f"Retrieving top {retrieve_k} chunks...")
    results = carol_col.query(
        query_texts=[question],
        n_results=retrieve_k,
        include=['documents', 'metadatas', 'distances']
    )
    
    documents = results['documents'][0]
    metadatas = results['metadatas'][0]
    doc_ids = results['ids'][0]
    initial_distances = results['distances'][0]
    
    # IMPROVEMENT #7: Rerank with cross-encoder
    print(f"Reranking with cross-encoder...")
    reranked = rerank_results(question, documents, top_k=use_k)
    
    # Get top reranked documents
    top_docs = [documents[idx] for score, idx in reranked]
    top_metadatas = [metadatas[idx] for score, idx in reranked]
    top_ids = [doc_ids[idx] for score, idx in reranked]
    top_scores = [score for score, idx in reranked]
    
    # Combine context from top reranked chunks
    context = "\n\n---\n\n".join(top_docs)
    
    # IMPROVEMENT #5: Better prompt with instructions for unknown cases
    question_prompt = f"""Answer the question based on the context below. 

IMPORTANT INSTRUCTIONS:
- Only answer if the information is clearly stated in the context
- If the answer is not in the context, respond with: "I don't have enough information to answer this question."
- Be concise and specific
- Use information directly from the context

CONTEXT:
{context}

QUESTION: {question}

ANSWER:"""
    
    # Generate answer
    print(f"Generating answer with {model_name}...\n")
    enc_prompt = tokenizer(
        question_prompt, 
        return_tensors='pt',
        max_length=1024,
        truncation=True
    )
    
    enc_answer = model.generate(
        enc_prompt.input_ids,
        max_length=100,
        num_beams=4,
        early_stopping=True
    )
    
    answer = tokenizer.decode(enc_answer[0], skip_special_tokens=True)
    
    # IMPROVEMENT #8: Show sources with chunk IDs
    sources = []
    if show_sources:
        print("="*80)
        print(f"ANSWER: {answer}")
        print("="*80)
        print(f"\nSOURCES (Top {use_k} chunks after reranking):\n")
        
        for i, (doc_id, metadata, score, doc) in enumerate(zip(
            top_ids, top_metadatas, top_scores, top_docs
        )):
            source_info = {
                'chunk_id': metadata['chunk_id'],
                'doc_id': doc_id,
                'rerank_score': score,
                'text_preview': doc[:200] + "..."
            }
            sources.append(source_info)
            
            print(f"{i+1}. Chunk #{metadata['chunk_id']} (ID: {doc_id})")
            print(f"   Rerank Score: {score:.4f}")
            print(f"   Preview: {doc[:150]}...")
            print()
    
    return {
        'answer': answer,
        'sources': sources,
        'num_chunks_retrieved': retrieve_k,
        'num_chunks_used': use_k
    }

## Test the Improved System

In [14]:
# Test question 1: Simple factual
result = answer_question_improved(
    "What is the name of Bob Cratchit's youngest son who is ill?",
    retrieve_k=10,
    use_k=5
)

Question: What is the name of Bob Cratchit's youngest son who is ill?

Retrieving top 10 chunks...
Reranking with cross-encoder...
Generating answer with google/flan-t5-large...

ANSWER: Be concise and specific

SOURCES (Top 5 chunks after reranking):

1. Chunk #235 (ID: 850e1a95)
   Rerank Score: -1.8312
   Preview: “Never, father!” cried they all.

“And I know,” said Bob, “I know, my dears, that when we recollect how patient and how mild he was; although he was a...

2. Chunk #230 (ID: 9e7c7abd)
   Rerank Score: -2.0635
   Preview: She hurried out to meet him; and little Bob in his comforter﻿—he had need of it, poor fellow﻿—came in. His tea was ready for him on the hob, and they ...

3. Chunk #236 (ID: 72d1b038)
   Rerank Score: -2.4814
   Preview: “No, never, father!” they all cried again.

“I am very happy,” said little Bob, “I am very happy!”

Mrs. Cratchit kissed him, his daughters kissed him...

4. Chunk #142 (ID: e54e0bf8)
   Rerank Score: -2.7188
   Preview: “We’d a deal of wo

In [15]:
# Test question 2: Requires inference
result = answer_question_improved(
    "Who was Scrooge's deceased business partner?",
    retrieve_k=10,
    use_k=5
)

Question: Who was Scrooge's deceased business partner?

Retrieving top 10 chunks...
Reranking with cross-encoder...
Generating answer with google/flan-t5-large...

ANSWER: I don't have enough information to answer the question.

SOURCES (Top 5 chunks after reranking):

1. Chunk #18 (ID: 54a8ebd7)
   Rerank Score: 3.1445
   Preview: This lunatic, in letting Scrooge’s nephew out, had let two other people in. They were portly gentlemen, pleasant to behold, and now stood, with their ...

2. Chunk #3 (ID: e2b05587)
   Rerank Score: 3.0446
   Preview: Mind! I don’t mean to say that I know of my own knowledge, what there is particularly dead about a doornail. I might have been inclined, myself, to re...

3. Chunk #19 (ID: f002351e)
   Rerank Score: 2.9361
   Preview: “Mr. Marley has been dead these seven years,” Scrooge replied. “He died seven years ago, this very night.”

“We have no doubt his liberality is well r...

4. Chunk #198 (ID: 299ed78d)
   Rerank Score: -0.0877
   Preview: The Spir

In [16]:
# Test question 3: Multiple parts
result = answer_question_improved(
    "Who was Scrooge engaged to in his youth, and why did she leave him?",
    retrieve_k=10,
    use_k=7  # Use more chunks for complex questions
)

Question: Who was Scrooge engaged to in his youth, and why did she leave him?

Retrieving top 10 chunks...
Reranking with cross-encoder...
Generating answer with google/flan-t5-large...

ANSWER: Be concise and specific

SOURCES (Top 7 chunks after reranking):

1. Chunk #75 (ID: 753bc4dd)
   Rerank Score: -0.1868
   Preview: “These are but shadows of the things that have been,” said the Ghost. “They have no consciousness of us.”

The jocund travellers came on; and as they ...

2. Chunk #97 (ID: dd0fb5a4)
   Rerank Score: -0.2189
   Preview: When the clock struck eleven, this domestic ball broke up. Mr. and Mrs. Fezziwig took their stations, one on either side the door, and, shaking hands ...

3. Chunk #112 (ID: 2e96149c)
   Rerank Score: -0.2488
   Preview: And now Scrooge looked on more attentively than ever, when the master of the house, having his daughter leaning fondly on him, sat down with her and h...

4. Chunk #15 (ID: 46f949e5)
   Rerank Score: -0.4558
   Preview: “But why?” cr

In [17]:
# Test question 4: Testing "unknown" handling
result = answer_question_improved(
    "What is Scrooge's favorite color?",  # Not in the book
    retrieve_k=10,
    use_k=5
)

Question: What is Scrooge's favorite color?

Retrieving top 10 chunks...
Reranking with cross-encoder...
Generating answer with google/flan-t5-large...

ANSWER: I don't have enough information to answer the question.

SOURCES (Top 5 chunks after reranking):

1. Chunk #124 (ID: dcfa51bf)
   Rerank Score: -0.5076
   Preview: Scrooge entered timidly, and hung his head before this Spirit. He was not the dogged Scrooge he had been; and though the Spirit’s eyes were clear and ...

2. Chunk #39 (ID: 43eb3f5b)
   Rerank Score: -1.0077
   Preview: “It’s humbug still!” said Scrooge. “I won’t believe it.”

His colour changed, though, when, without a pause, it came on through the heavy door and pas...

3. Chunk #126 (ID: ec5f8721)
   Rerank Score: -2.9501
   Preview: “A tremendous family to provide for,” muttered Scrooge.

The Ghost of Christmas Present rose.

“Spirit,” said Scrooge submissively, “conduct me where ...

4. Chunk #249 (ID: 992848c9)
   Rerank Score: -3.7638
   Preview: Running to th

## Batch Process Questions

In [19]:
# Load questions from file if available
questions = [
    "What is the name of Scrooge's underpaid clerk?",
    "Who was Scrooge's deceased business partner?",
    "Who was Scrooge engaged to in his youth, and why did she leave him?",
    "What is the name of Bob Cratchit's youngest son who is ill?",
    "What does Scrooge see written on the gravestone that frightens him into changing his ways?",
    "What is Scrooge's response when his nephew Fred invites him to Christmas dinner at the beginning of the story?",
    "What specific, generous act does Scrooge perform for the Cratchit family on Christmas morning?"
]

print("Processing all questions...\n")
print("="*80)

for i, q in enumerate(questions, 1):
    print(f"\n{'='*80}")
    print(f"Question {i}/{len(questions)}")
    print("="*80)
    result = answer_question_improved(q, retrieve_k=10, use_k=5, show_sources=True)
    print("\n")

Processing all questions...


Question 1/7
Question: What is the name of Scrooge's underpaid clerk?

Retrieving top 10 chunks...
Reranking with cross-encoder...
Generating answer with google/flan-t5-large...

ANSWER: Only answer if the information is clearly stated in the context.

SOURCES (Top 5 chunks after reranking):

1. Chunk #9 (ID: 1692abf5)
   Rerank Score: 0.7962
   Preview: The door of Scrooge’s counting house was open, that he might keep his eye upon his clerk, who in a dismal little cell beyond, a sort of tank, was copy...

2. Chunk #28 (ID: 03f5311b)
   Rerank Score: 0.2324
   Preview: The clerk smiled faintly.

“And yet,” said Scrooge, “you don’t think me ill used when I pay a day’s wages for no work.”

The clerk observed that it wa...

3. Chunk #16 (ID: ab95cb00)
   Rerank Score: -2.8719
   Preview: “Good afternoon,” said Scrooge.

“I want nothing from you; I ask nothing of you; why cannot we be friends?”

“Good afternoon!” said Scrooge.

“I am so...

4. Chunk #258 (ID: 

## Summary of Improvements

### Performance Comparison

| Aspect | Original | Improved | Impact |
|--------|----------|----------|--------|
| Chunking overlap | 50 chars (5%) | 512 chars (50%) | Better context preservation |
| Storage | Ephemeral (lost on restart) | Persistent (saved to disk) | No re-embedding needed |
| Retrieval | 3 chunks | 10 chunks → reranked to 5 | More comprehensive context |
| Question processing | 2-step reformulation | Direct search | Simpler, faster, more accurate |
| Model size | FLAN-T5-base (250M) | FLAN-T5-large (780M) | Better reasoning |
| Reranking | None | Cross-encoder | Better relevance scoring |
| Prompt quality | Basic | Instructive with unknown handling | Better answers |
| Source attribution | None | Chunk IDs + metadata | Verifiable answers |

### Expected Results

The improved system should:
- Give more accurate answers
- Handle complex multi-part questions better
- Correctly identify when information is not available
- Provide source citations for verification
- Run faster after first initialization (persistent storage)
- Be more production-ready

### Further Improvements (Optional)

If you want to go even further:
1. Use a modern LLM (Claude, GPT-4, Llama) instead of FLAN-T5
2. Add query expansion (generate multiple query variations)
3. Implement hybrid search (keyword + semantic)
4. Add semantic chunking (split on topics, not characters)
5. Use a better embedding model (e.g., OpenAI ada-002, Cohere embed-v3)
6. Add metadata filtering (by chapter, character, etc.)
7. Implement multi-hop reasoning for complex questions
8. Add answer validation/verification step